下载数据的命令

python collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN  --delay 0.1 --interval 1d

# download from eastmoney.com
python collector.py download_data --source_dir ~/.qlib/fund_data/source/cn_1d --region CN --start 2020-11-01 --end 2020-11-10 --delay 0.1 --interval 1d

# normalize
python collector.py normalize_data --source_dir ~/.qlib/fund_data/source/cn_1d --normalize_dir ~/.qlib/fund_data/source/cn_1d_nor --region CN --interval 1d --date_field_name FSRQ

# dump data
cd qlib/scripts

python dump_bin.py dump_all --csv_path ~/.qlib/fund_data/source/cn_1d_nor --qlib_dir ~/.qlib/qlib_data/cn_fund_data --freq day --date_field_name FSRQ --include_fields DWJZ,LJJZ

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from read_and_write_fund_code import *

# Step1： 读出来所有的数据

In [2]:
import qlib
from qlib.data import D

qlib.init(provider_uri="~/.qlib/qlib_data/cn_fund_data")
df = D.features(D.instruments(market="all"), ["$DWJZ", "$LJJZ"], freq="day")

[86714:MainThread](2021-07-10 22:59:26,382) INFO - qlib.Initialization - [config.py:276] - default_conf: client.
[86714:MainThread](2021-07-10 22:59:26,383) WARNING - qlib.Initialization - [config.py:291] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[86714:MainThread](2021-07-10 22:59:26,508) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[86714:MainThread](2021-07-10 22:59:26,509) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/wangfan/.qlib/qlib_data/cn_fund_data


In [3]:
df.head()

$DWJZ  $LJJZ
instrument datetime                
000001     2005-01-04  0.995  1.115
           2005-01-05  0.998  1.118
           2005-01-06  0.991  1.111
           2005-01-07  0.989  1.109
           2005-01-10  0.992  1.112

In [4]:
df.tail()

$DWJZ   $LJJZ
instrument datetime                  
980003     2021-07-05  1.4746  1.4746
           2021-07-06  1.4749  1.4749
           2021-07-07  1.4751  1.4751
           2021-07-08  1.4757  1.4757
           2021-07-09  1.4759  1.4759

In [5]:
df = df.reset_index()

In [6]:
df_new = df.copy()

# Step2 数据加工

In [7]:
df_new["LJJZ_1d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-1)
df_new["LJJZ_5d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-5)
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-480)
df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(-720)
df_new.index = range(len(df_new))
df_new["y_1d"]=100 * (df_new["LJJZ_1d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_5d"]=100 * (df_new["LJJZ_5d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_20d"]=100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_60d"]=100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_120d"]=100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_240d"]=100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_480d"]=100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new["y_720d"]=100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["$LJJZ"]
df_new = df_new.drop(columns = ['LJJZ_1d','LJJZ_5d','LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d','LJJZ_720d'])

In [8]:
df_new["LJJZ_20d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(20)
df_new["LJJZ_60d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(60)
df_new["LJJZ_120d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(120)
df_new["LJJZ_240d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(240)
df_new["LJJZ_480d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(480)
df_new["LJJZ_720d"] = df_new.groupby(['instrument'])["$LJJZ"].shift(720)
df_new.index = range(len(df_new))


df_new["his_20d"]=-100 * (df_new["LJJZ_20d"]- df_new["$LJJZ"])/df_new["LJJZ_20d"]
df_new["his_60d"]=-100 * (df_new["LJJZ_60d"]- df_new["$LJJZ"])/df_new["LJJZ_60d"]
df_new["his_120d"]=-100 * (df_new["LJJZ_120d"]- df_new["$LJJZ"])/df_new["LJJZ_120d"]
df_new["his_240d"]=-100 * (df_new["LJJZ_240d"]- df_new["$LJJZ"])/df_new["LJJZ_240d"]
df_new["his_480d"]=-100 * (df_new["LJJZ_480d"]- df_new["$LJJZ"])/df_new["LJJZ_480d"]
df_new["his_720d"]=-100 * (df_new["LJJZ_720d"]- df_new["$LJJZ"])/df_new["LJJZ_720d"]
df_new = df_new.drop(columns = ['LJJZ_20d','LJJZ_60d','LJJZ_120d','LJJZ_240d','LJJZ_480d','LJJZ_720d'])

In [9]:
df_train = df_new[df_new['y_720d'].notnull()]
df_test = df_new[df_new['y_720d'].isnull()]

In [10]:
df_train = df_train.dropna()

In [11]:
df_train["rank_20d"] = df_train.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
df_train["rank_60d"] = df_train.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
df_train["rank_120d"] = df_train.groupby(['datetime'])["his_120d"].rank(method='min',ascending=False)
df_train["rank_240d"] = df_train.groupby(['datetime'])["his_240d"].rank(method='min',ascending=False)
df_train["rank_480d"] = df_train.groupby(['datetime'])["his_480d"].rank(method='min',ascending=False)
df_train["rank_720d"] = df_train.groupby(['datetime'])["his_720d"].rank(method='min',ascending=False)
df_train.index = range(len(df_train))
# rank越小涨幅越大

In [12]:
w1, w2, w3, w4, w5, w6 = 6, 5, 4, 3, 2, 1

In [13]:
df_train["average_rank"] = df_train["rank_20d"] * w1 + df_train["rank_60d"] * w2 + df_train["rank_120d"] * w3 + df_train["rank_240d"] * w4 + df_train["rank_480d"] * w5 + df_train["rank_720d"] * w6
#df_train["average_rank"] = df_train["rank_120d"] + df_train["rank_240d"] + df_train["rank_480d"] + df_train["rank_720d"]
df_train["rank_of_average_rank"] = df_train.groupby(['datetime'])["average_rank"].rank(method='min')
df_train.index = range(len(df_train))
# rank越小，平均排名越高

In [14]:
df_train[(df_train['rank_of_average_rank']==1)&(df_train['datetime']=='2016-05-11')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
1146754,320017,2016-05-11,1814.0,1.0


In [15]:
df_train[(df_train['rank_of_average_rank']==10)&(df_train['datetime']=='2016-05-11')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
528765,160215,2016-05-11,4058.0,10.0


In [16]:
df_test["rank_20d"] = df_test.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
df_test["rank_60d"] = df_test.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
df_test["rank_120d"] = df_test.groupby(['datetime'])["his_120d"].rank(method='min',ascending=False)
df_test["rank_240d"] = df_test.groupby(['datetime'])["his_240d"].rank(method='min',ascending=False)
df_test["rank_480d"] = df_test.groupby(['datetime'])["his_480d"].rank(method='min',ascending=False)
df_test["rank_720d"] = df_test.groupby(['datetime'])["his_720d"].rank(method='min',ascending=False)
df_test.index = range(len(df_test))

<ipython-input-16-aa3ebe0c304f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["rank_20d"] = df_test.groupby(['datetime'])["his_20d"].rank(method='min',ascending=False)
<ipython-input-16-aa3ebe0c304f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["rank_60d"] = df_test.groupby(['datetime'])["his_60d"].rank(method='min',ascending=False)
<ipython-input-16-aa3ebe0c304f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [17]:
df_test["average_rank"] = df_test["rank_20d"] * w1 + df_test["rank_60d"] * w2 + df_test["rank_120d"] * w3 + df_test["rank_240d"] * w4 + df_test["rank_480d"] * w5 + df_test["rank_720d"] * w6
df_test["rank_of_average_rank"] = df_test.groupby(['datetime'])["average_rank"].rank(method='min')
df_test.index = range(len(df_test))

<ipython-input-17-983948cffb03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["average_rank"] = df_test["rank_20d"] * w1 + df_test["rank_60d"] * w2 + df_test["rank_120d"] * w3 + df_test["rank_240d"] * w4 + df_test["rank_480d"] * w5 + df_test["rank_720d"] * w6
<ipython-input-17-983948cffb03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["rank_of_average_rank"] = df_test.groupby(['datetime'])["average_rank"].rank(method='min')


# Step 3 模拟回测

In [18]:
df_train.columns

Index(['instrument', 'datetime', '$DWJZ', '$LJJZ', 'y_1d', 'y_5d', 'y_20d',
       'y_60d', 'y_120d', 'y_240d', 'y_480d', 'y_720d', 'his_20d', 'his_60d',
       'his_120d', 'his_240d', 'his_480d', 'his_720d', 'rank_20d', 'rank_60d',
       'rank_120d', 'rank_240d', 'rank_480d', 'rank_720d', 'average_rank',
       'rank_of_average_rank'],
      dtype='object')

In [19]:
def simulation_240d(start_rank,end_rank,start_date='1900-01-01',end_date='2021-04-29',dingtou=False,frequency=5):
    if not dingtou:
        return np.mean(df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)&(df_train['datetime']>start_date)&(df_train['datetime']<end_date)]['y_240d'])

In [20]:
start_rank=11
end_rank=15
start_date='1900-01-01'
end_date='2021-04-29'
df_train[(df_train['rank_of_average_rank']>=start_rank)
                &(df_train['rank_of_average_rank']<=end_rank)
                 &(df_train['datetime']>start_date)
                 &(df_train['datetime']<end_date)]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
9,000001,2008-01-08,2.2670,3.4480,1.015084,2.233183,-2.871229,-11.397908,-15.081206,-23.810900,...,206.216690,212.036179,13.0,27.0,33.0,41.0,44.0,53.0,609.0,15.0
11,000001,2008-01-10,2.3390,3.5200,0.369319,-2.414774,-5.227273,-12.301140,-15.710226,-24.715904,...,215.695068,221.167908,14.0,14.0,33.0,37.0,43.0,51.0,534.0,14.0
12,000001,2008-01-11,2.3520,3.5330,0.198131,-1.981317,-6.057175,-14.690066,-16.161905,-24.568356,...,213.487137,223.831360,15.0,7.0,31.0,39.0,43.0,50.0,502.0,14.0
17,000001,2008-01-18,2.2820,3.4630,-2.252384,-1.414959,-4.187120,-15.737802,-16.142073,-23.678896,...,201.655045,210.304657,15.0,11.0,23.0,32.0,42.0,52.0,469.0,14.0
18,000001,2008-01-21,2.2040,3.3850,-3.161001,-1.358937,-3.190548,-13.825701,-13.648446,-21.004431,...,190.807541,203.315414,10.0,14.0,26.0,35.0,45.0,57.0,486.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761253,730001,2015-12-10,1.5770,2.3270,-0.859475,3.266017,-10.915339,-17.576277,-8.809623,-13.278901,...,116.666656,130.168137,59.0,155.0,399.0,20.0,46.0,127.0,3004.0,15.0
1766257,750005,2016-05-09,1.4750,1.4750,0.000000,0.067792,0.542374,4.000001,6.101697,7.925421,...,48.390343,46.039608,173.0,316.0,83.0,154.0,300.0,243.0,4255.0,14.0
1766898,762001,2016-02-17,1.6773,2.3073,-0.199369,1.304549,-0.988177,3.155200,9.378923,10.800496,...,106.858536,119.742874,299.0,195.0,248.0,14.0,10.0,30.0,3853.0,11.0
1766899,762001,2016-02-18,1.6727,2.3027,0.047773,-1.103060,0.047773,4.933340,8.737569,11.538623,...,104.575348,118.203362,121.0,231.0,269.0,12.0,9.0,26.0,3037.0,11.0


In [21]:
simulation_240d(1,10,'2012-01-01')

8.065358

In [22]:
simulation_240d(2,11,'2012-01-01')

8.1837845

In [23]:
simulation_240d(1,5,'2012-01-01')

8.105107

In [24]:
simulation_240d(11,20,'2012-01-01')

7.6779647

In [25]:
simulation_240d(11,15,'2012-01-01')

7.7376714

In [26]:
simulation_240d(21,30,'2012-01-01')

7.8843646

# 2013以后

In [27]:
simulation_240d(1,10,'2013-01-01')

8.340794

In [28]:
simulation_240d(1,10,'2014-01-01')

8.494433

In [29]:
simulation_240d(1,10,'2015-01-01')

-0.043738324

In [30]:
simulation_240d(1,10,'2016-01-01')

3.1351373

In [31]:
simulation_240d(1,10,'2017-01-01')

0.18076365

In [32]:
simulation_240d(1,10,'2018-01-01')

-5.674363

In [33]:
simulation_240d(1,10,'2019-01-01')

nan

# 定投

# Step 4 预测未来最优潜力组合

In [34]:
df_test[(df_test['instrument'] == '002943')&(df_test['datetime'] == '2021-05-25')]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
1491375,002943,2021-05-25,2.8372,3.0775,0.854588,5.0593,4.532897,NaN,NaN,NaN,...,163.169159,177.903214,947.0,377.0,35.0,67.0,175.0,54.0,8312.0,109.0


In [35]:
df_test[(df_test['rank_of_average_rank']==10)&(df_test['datetime']=='2021-04-29')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
327561,000684,2021-04-29,3349.0,10.0


In [36]:
df_test[(df_test['rank_of_average_rank']==1)&(df_test['datetime']=='2021-04-29')][['instrument','datetime','average_rank','rank_of_average_rank']]

,instrument,datetime,average_rank,rank_of_average_rank
2575985,005296,2021-04-29,2080.0,1.0


In [53]:
def get_list(start_rank,end_rank,date='2021-04-29'): 
    df_left = df_test[(df_test['rank_of_average_rank']>=start_rank)&(df_test['rank_of_average_rank']<=end_rank)&(df_test['datetime']==date)][['instrument','datetime','average_rank','rank_of_average_rank']].sort_values(by=['rank_of_average_rank'])
    df_fund_code = read_fund_code_from_DB()
    return pd.merge(df_left, df_fund_code, how='left', left_on='instrument', right_on='code')

In [54]:
get_list(11,20,'2021-04-29')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,000831,2021-04-29,3644.0,11.0,605,000831,GYYLBJGP,工银医疗保健股票,股票型
1,001365,2021-04-29,3886.0,12.0,1035,001365,DCZXHBLHPZHH,大成正向回报灵活配置混合,混合型-灵活
2,168203,2021-04-29,4288.0,13.0,11417,168203,ZRGZGTHYZSLOF,中融国证钢铁行业指数(LOF),指数型-股票
3,519171,2021-04-29,5033.0,14.0,12505,519171,PYASYLJKHH,浦银安盛医疗健康混合,混合型-灵活
4,002708,2021-04-29,5772.0,15.0,2057,002708,DMJKCYHH,大摩健康产业混合,混合型-偏股
5,003291,2021-04-29,5840.0,16.0,2479,003291,XDAYJKZGHH,信达澳银健康中国混合,混合型-灵活
6,377150,2021-04-29,6001.0,17.0,11784,377150,STMGJKPZSHHH,上投摩根健康品质生活混合,混合型-偏股
7,501038,2021-04-29,6452.0,18.0,11965,501038,YHMZDCLDQKFHH,银华明择多策略定期开放混合,混合型-灵活
8,005004,2021-04-29,7234.0,19.0,3640,005004,JYPZSJHH,交银品质升级混合,混合型-偏股
9,004868,2021-04-29,7267.0,20.0,3527,004868,JYGXYHHH,交银股息优化混合,混合型-偏股


In [55]:
get_list(11,20,'2021-04-29')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,000831,2021-04-29,3644.0,11.0,605,000831,GYYLBJGP,工银医疗保健股票,股票型
1,001365,2021-04-29,3886.0,12.0,1035,001365,DCZXHBLHPZHH,大成正向回报灵活配置混合,混合型-灵活
2,168203,2021-04-29,4288.0,13.0,11417,168203,ZRGZGTHYZSLOF,中融国证钢铁行业指数(LOF),指数型-股票
3,519171,2021-04-29,5033.0,14.0,12505,519171,PYASYLJKHH,浦银安盛医疗健康混合,混合型-灵活
4,002708,2021-04-29,5772.0,15.0,2057,002708,DMJKCYHH,大摩健康产业混合,混合型-偏股
5,003291,2021-04-29,5840.0,16.0,2479,003291,XDAYJKZGHH,信达澳银健康中国混合,混合型-灵活
6,377150,2021-04-29,6001.0,17.0,11784,377150,STMGJKPZSHHH,上投摩根健康品质生活混合,混合型-偏股
7,501038,2021-04-29,6452.0,18.0,11965,501038,YHMZDCLDQKFHH,银华明择多策略定期开放混合,混合型-灵活
8,005004,2021-04-29,7234.0,19.0,3640,005004,JYPZSJHH,交银品质升级混合,混合型-偏股
9,004868,2021-04-29,7267.0,20.0,3527,004868,JYGXYHHH,交银股息优化混合,混合型-偏股


In [56]:
get_list(11,20,'2021-05-25')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,004075,2021-05-25,2427.0,11.0,2989,004075,JYYYCXGP,交银医药创新股票,股票型
1,003095,2021-05-25,2505.0,12.0,2334,003095,ZOYLJKHHA,中欧医疗健康混合A,混合型-偏股
2,000831,2021-05-25,2614.0,13.0,605,000831,GYYLBJGP,工银医疗保健股票,股票型
3,003096,2021-05-25,2748.0,14.0,2335,003096,ZOYLJKHHC,中欧医疗健康混合C,混合型-偏股
4,000452,2021-05-25,2780.0,15.0,311,000452,NFYYBJLHPZHH,南方医药保健灵活配置混合,混合型-灵活
5,003834,2021-05-25,2974.0,16.0,2847,003834,HXNYGXGP,华夏能源革新股票,股票型
6,002708,2021-05-25,2975.0,17.0,2057,002708,DMJKCYHH,大摩健康产业混合,混合型-偏股
7,002083,2021-05-25,3102.0,18.0,1594,002083,XHXDLLHPZHHA,新华鑫动力灵活配置混合A,混合型-灵活
8,002084,2021-05-25,3139.0,19.0,1595,002084,XHXDLLHPZHHC,新华鑫动力灵活配置混合C,混合型-灵活
9,005235,2021-05-25,3157.0,20.0,3826,005235,YHSPYLLHGPFQSA,银华食品饮料量化股票发起式A,股票型


In [57]:
get_list(1,10,'2021-05-25')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,002408,2021-05-25,947.0,1.0,1831,002408,ZXJTYGHHA,中信建投医改混合A,混合型-灵活
1,000209,2021-05-25,1108.0,2.0,164,000209,XCXXCYHH,信诚新兴产业混合,混合型-偏股
2,001298,2021-05-25,1336.0,3.0,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
3,001717,2021-05-25,1448.0,4.0,1303,001717,GYQYYLGPA,工银前沿医疗股票A,股票型
4,005296,2021-05-25,1539.0,5.0,3876,005296,NHFCHHA,南华丰淳混合A,混合型-偏股
5,000684,2021-05-25,1570.0,6.0,487,000684,CSYLJKCYLHPZHH,长盛养老健康产业灵活配置混合,混合型-灵活
6,005297,2021-05-25,1641.0,7.0,3877,005297,NHFCHHC,南华丰淳混合C,混合型-偏股
7,002300,2021-05-25,2321.0,8.0,1751,002300,CSYLHYLHPZGP,长盛医疗行业量化配置股票,股票型
8,005164,2021-05-25,2365.0,9.0,3766,005164,FRFJHHA,富荣福锦混合A,混合型-偏股
9,005165,2021-05-25,2409.0,10.0,3767,005165,FRFJHHC,富荣福锦混合C,混合型-偏股


In [67]:
get_list(1,30,'2021-05-25')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,002408,2021-05-25,947.0,1.0,1831,002408,ZXJTYGHHA,中信建投医改混合A,混合型-灵活
1,000209,2021-05-25,1108.0,2.0,164,000209,XCXXCYHH,信诚新兴产业混合,混合型-偏股
2,001298,2021-05-25,1336.0,3.0,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
3,001717,2021-05-25,1448.0,4.0,1303,001717,GYQYYLGPA,工银前沿医疗股票A,股票型
4,005296,2021-05-25,1539.0,5.0,3876,005296,NHFCHHA,南华丰淳混合A,混合型-偏股
5,000684,2021-05-25,1570.0,6.0,487,000684,CSYLJKCYLHPZHH,长盛养老健康产业灵活配置混合,混合型-灵活
6,005297,2021-05-25,1641.0,7.0,3877,005297,NHFCHHC,南华丰淳混合C,混合型-偏股
7,002300,2021-05-25,2321.0,8.0,1751,002300,CSYLHYLHPZGP,长盛医疗行业量化配置股票,股票型
8,005164,2021-05-25,2365.0,9.0,3766,005164,FRFJHHA,富荣福锦混合A,混合型-偏股
9,005165,2021-05-25,2409.0,10.0,3767,005165,FRFJHHC,富荣福锦混合C,混合型-偏股


In [60]:
df_test[(df_test['instrument'] == '000828')&(df_test['datetime'] =='2021-06-25')]

,instrument,datetime,$DWJZ,$LJJZ,y_1d,y_5d,y_20d,y_60d,y_120d,y_240d,...,his_480d,his_720d,rank_20d,rank_60d,rank_120d,rank_240d,rank_480d,rank_720d,average_rank,rank_of_average_rank
382061,000828,2021-06-25,2.895,3.115,1.444625,4.847513,NaN,NaN,NaN,NaN,...,187.361633,209.642136,48.0,13.0,99.0,55.0,71.0,66.0,1122.0,4.0


In [64]:
get_list(1,30,'2021-06-08')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,000209,2021-06-08,561.0,1.0,164,000209,XCXXCYHH,信诚新兴产业混合,混合型-偏股
1,002190,2021-06-08,646.0,2.0,1669,002190,NYXNYZT,农银新能源主题,混合型-灵活
2,003834,2021-06-08,696.0,3.0,2847,003834,HXNYGXGP,华夏能源革新股票,股票型
3,002083,2021-06-08,713.0,4.0,1594,002083,XHXDLLHPZHHA,新华鑫动力灵活配置混合A,混合型-灵活
4,002084,2021-06-08,742.0,5.0,1595,002084,XHXDLLHPZHHC,新华鑫动力灵活配置混合C,混合型-灵活
5,000828,2021-06-08,1060.0,6.0,602,000828,TDZXJY,泰达转型机遇,股票型
6,005885,2021-06-08,1418.0,7.0,4386,005885,JYXXCYGPC,金鹰信息产业股票C,股票型
7,003853,2021-06-08,1440.0,8.0,2863,003853,JYXXCYGPA,金鹰信息产业股票A,股票型
8,002408,2021-06-08,1747.0,9.0,1831,002408,ZXJTYGHHA,中信建投医改混合A,混合型-灵活
9,003567,2021-06-08,1883.0,10.0,2679,003567,HXHYJQHH,华夏行业景气混合,混合型-偏股


In [63]:
get_list(1,30,'2021-07-08')

,instrument,datetime,average_rank,rank_of_average_rank,index,code,ename,cname,type
0,001298,2021-07-08,382.0,1.0,980,001298,JYMZXXHH,金鹰民族新兴混合,混合型-灵活
1,005927,2021-07-08,708.0,2.0,4419,005927,CJHXXNYQCGPA,创金合信新能源汽车股票A,股票型
2,000209,2021-07-08,759.0,3.0,164,000209,XCXXCYHH,信诚新兴产业混合,混合型-偏股
3,005928,2021-07-08,773.0,4.0,4420,005928,CJHXXNYQCGPC,创金合信新能源汽车股票C,股票型
4,000689,2021-07-08,965.0,5.0,491,000689,QHKYXJJHH,前海开源新经济混合,混合型-灵活
5,005669,2021-07-08,965.0,5.0,4197,005669,QHKYGYSYGP,前海开源公用事业股票,股票型
6,000828,2021-07-08,973.0,7.0,602,000828,TDZXJY,泰达转型机遇,股票型
7,004925,2021-07-08,984.0,8.0,3575,004925,CXDTHBHYLHGP,长信低碳环保行业量化股票,股票型
8,501057,2021-07-08,1046.0,9.0,11980,501057,HTFZZXNYQCA,汇添富中证新能源汽车A,指数型-股票
9,002256,2021-07-08,1052.0,10.0,1717,002256,JXHYYXHHFQS,金信行业优选混合发起式,混合型-灵活
